### Welcome to Week 6 Day 3!

Let's experiment with a bunch more MCP Servers

In [15]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

### The first type of MCP Server: runs locally, everything local

Here's a really interesting one: a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


In [2]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/ed.db"}}

# start the mcp server as a subprocess and connect via stdio (stdin / stdout)
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:

    # ask the mcp server: what tools do you provide
    # this sends an mcp json-rpc request: tool/list
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', title='Create new entities with observations', description='Create new entities with observations', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities'], '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=None, meta=None),
 Tool(name='search_nodes', title='Search for entities and their relations using text search with relevance ranking', description='Search for entities and their relations using text search with relevance ranking', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string'}, 'limit': {'type': 'number'}}, 'required': ['query'], '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, 

In [3]:
# system-level instruction to the llm
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."

# user message that will be processed by the llm
request = "My name's Ed. I'm an LLM engineer. I'm teaching a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."

model = "gpt-4.1-mini"

In [4]:
# start the mcp server again and connect it
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:

    # create an agent that has access to mcp tools
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])

    # start tracing/logging for debugging
    with trace("conversation"):

        # run the agent on the user request
        result = await Runner.run(agent, request)

    # display the final llm output
    display(Markdown(result.final_output))

Nice to meet you, Ed! It's great to hear about your course on AI Agents and the MCP protocol. If you'd like, I can help you organize your course material, explain concepts, or provide more information on AI agents and related topics. How can I assist you further?

In [5]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Ed. What do you know about me?")
    display(Markdown(result.final_output))

You are Ed, an LLM engineer. Additionally, you are teaching a course about AI Agents. Would you like to add or update any information about yourself?

### Check the trace:

https://platform.openai.com/traces

### The 2nd type of MCP server - runs locally, calls a web service

### Brave Search - apologies - this will need another API key! But it's free again.

https://brave.com/search/api/

Set up your account, and put your key in the .env under `BRAVE_API_KEY`

In [8]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', title=None, description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, outputSchema=None, icons=None, annotations=None, meta=None),
 Tool(name='brave_local_search', title=None, description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, re

In [11]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Micron stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [12]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Here’s a summary of the latest news on Micron stock as of January 23, 2026:

1. **Strong Start in 2026**: Micron Technology has had a successful beginning to the year. Analysts believe that the stock price could potentially triple within the year, with optimistic earnings projections of around $37.29 per share over the next four quarters.

2. **Price Target Increases**: The stock saw a notable surge earlier in January when Bernstein raised its price target by 20%, reflecting increased investor confidence. Argus also revised their price target to $320, indicating solid backing from analysts.

3. **AI Memory Demand**: A surge in demand for AI-related memory solutions has been a significant factor driving up Micron's stock price. The stock rose approximately 16.67% in the first week of January, closing at $343.43.

4. **Recent Performance**: As of mid-January, Micron's stock closed at $362.75, nearing its 52-week high, suggesting a strong accumulation phase among investors.

Overall, the outlook for Micron is positive, driven by robust demand in the AI sector and strong analyst support.

### As usual, check out the trace:

https://platform.openai.com/traces

## And now the third type: running remotely

It's actually really hard to find a "remote MCP server" aka "hosted MCP server" aka "managed MCP server".

It's not a common model for using or sharing MCP servers, and there isn't a standard way to discover remote MCP servers.

Anthropic lists some remote MCP servers, but these are for paid applications with business users:

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

CloudFlare has tooling for you to create and deploy your own remote MCP servers, but this does not seem to be a common practice:

https://developers.cloudflare.com/agents/guides/remote-mcp-server/


# And back to the 2nd type: the Polygon.io MCP Server

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">PLEASE READ!!-</h2>
            <span style="color:#ff7800;">This service for financial market data has both a FREE plan and a PAID plan, and we can use either depending on your appetite.
            </span>
        </td>
    </tr>
</table>

## NEW SECTION: Introducing polygon.io

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

First, read up on polygon.io on their excellent website, including looking at their pricing:

https://polygon.io

### Polygon.io Part 1: Polygon.io free service (the paid will be totally optional, of course!)

1. Please sign up for polygon.io (top right)  
2. Once signed in, please select "Keys" in the left hand navigation
3. Press the blue "New Key" button
4. Copy the key name
5. Edit your .env file and add the row:

`POLYGON_API_KEY=xxxx`

In [3]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [4]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=248.35, high=251, low=248.15, open=249.2, timestamp=1769115600000, volume=39706430.0, vwap=249.2648)

### Wrapped into a python module that caches end of day prices

I've made a python module `market.py` that uses this API to look up share prices.

But the free API is quite heavily rate limited - so I've been a bit sneaky; when you ask for a share price, this function retrieves the entire end-of-day equity market, and caches it in our database.


In [5]:
from market import get_share_price
get_share_price("AAPL")

248.35

In [6]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

248.35

### And I've made this into an MCP Server

Just as we did with accounts.py; see `market_server.py`

In [7]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', title=None, description='This tool provides the current price of the given stock symbol.\n\n    Args:\n        symbol: the symbol of the stock\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'lookup_share_priceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None)]

### Let's try it out!

Hopefully gpt-4o-mini is smart enough to know that the symbol for Apple is AAPL

In [ ]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Apple (AAPL) is $248.35.

[non-fatal] Tracing client error 400: {
  "error": {
    "message": "Invalid type for 'data[1].span_data.result': expected an array of strings, but got null instead.",
    "type": "invalid_request_error",
    "param": "data[1].span_data.result",
    "code": "invalid_type"
  }
}


## Polygon.io Part 2: Paid Plan - Totally Optional!

If you are interested, you can subscribe to the monthly plan to get more up to date market data, and unlimited API calls.

If you do wish to do this, then it also makes sense to use the full MCP server that Polygon.io has released, to take advantage of all their functionality.



In [13]:

params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools


[Tool(name='get_aggs', title=None, description='\n    List aggregate bars for a ticker over a given date range in custom time window sizes.\n    ', inputSchema={'properties': {'ticker': {'title': 'Ticker', 'type': 'string'}, 'multiplier': {'title': 'Multiplier', 'type': 'integer'}, 'timespan': {'title': 'Timespan', 'type': 'string'}, 'from_': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'From'}, 'to': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'To'}, 'adjusted': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': None, 'title': 'Adjusted'}, 'sort': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Sort'}, 'limit': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Limit'}, 'params': {'anyOf': [{'additionalProperties': True, 'typ

### Wow that's a lot of tools!

Let's try them out - hopefully the sheer number of tools doesn't overwhelm gpt-4o-mini!

With the $29 monthly plan, we don't have access to some of the APIs, so I've needed to specify which APIs can be called.

If you've splashed out on a bigger plan, feel free to remove my extra constraint..

In [14]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

I am currently unable to access the latest Apple (AAPL) share price due to data access restrictions. Is there anything else you would like to know or another way I can assist you?

## Setting up your .env file

If you do decide to have a paid plan, please add this to your .env file to indicate:

`POLYGON_PLAN=paid`

And if you decide to go all the way for the realtime API, then please do:

`POLYGON_PLAN=realtime`

In [ ]:
load_dotenv(override=True)

polygon_plan = os.getenv("POLYGON_PLAN")
is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

if is_paid_polygon:
    print("You've chosen to subscribe to the paid Polygon plan, so the code will look at prices on a 15 min delay")
elif is_realtime_polygon:
    print("Wowzer - you've chosen to subscribe to the realtime Polygon plan, so the code will look at realtime prices")
else:
    print("According to your .env file, you've chosen to subscribe to the free Polygon plan, so the code will look at EOD prices")

## And that's it for today!

I've removed the part of this lab that uses the "Financial Datasets" mcp server, because it's inferior - more expensive with fewer APIs.

And this way we get to use the same provider for Free and Paid APIs.

But if you want to see the code, just look in the git history for a prior version.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Explore MCP server marketplaces and integrate your own, using all 3 approaches.
            </span>
        </td>
    </tr>
</table>

In [16]:
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display

load_dotenv(override=True)

params = {
    "command" : "npx",
    "args" : ["-y", "@modelcontextprotocol/server-github"],
    "env" : {
        "GITHUB_TOKEN": os.getenv("GITHUB_TOKEN")
    }
}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    tools = await server.list_tools()

tools

[Tool(name='create_or_update_file', title=None, description='Create or update a single file in a GitHub repository', inputSchema={'type': 'object', 'properties': {'owner': {'type': 'string', 'description': 'Repository owner (username or organization)'}, 'repo': {'type': 'string', 'description': 'Repository name'}, 'path': {'type': 'string', 'description': 'Path where to create/update the file'}, 'content': {'type': 'string', 'description': 'Content of the file'}, 'message': {'type': 'string', 'description': 'Commit message'}, 'branch': {'type': 'string', 'description': 'Branch to create/update the file in'}, 'sha': {'type': 'string', 'description': 'SHA of the file being replaced (required when updating existing files)'}}, 'required': ['owner', 'repo', 'path', 'content', 'message', 'branch'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=None, meta=None),
 Tool(name='search_repositories', title=None, des

In [18]:
instructions = "You are a software engineer. Use GitHub tools to inspect repositories and explain code clearly."

request = "Explore the repo microsoft/vscode: Visual Studio Code and explain its architecture in simple terms"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="github-agent",
        instructions=instructions,
        model=model,
        mcp_servers=[mcp_server]
    )

    with trace("conversation"):
        result = await Runner.run(agent, request)

display(Markdown(result.final_output))

The README of the microsoft/vscode repo describes the following about the project and its architecture context:

- This repo, called "Code - OSS," is where Microsoft develops the core open-source Visual Studio Code product along with the community. The source is under the MIT license.
- Visual Studio Code is a code editor combined with developer essentials for editing, building, and debugging.
- It includes code editing, navigation, understanding, lightweight debugging, rich extensibility, and lightweight integration with existing tools.
- VS Code is updated monthly with new features and bug fixes and is available for Windows, macOS, and Linux.
- The product you use as Visual Studio Code includes Microsoft-specific customizations on top of this open-source core.
- The repo also contains built-in extensions (in the "extensions" folder), such as language grammars, snippets, and rich language support.
- There are also separate related repos for core components and extensions like debug adapters.
- For development, the repo supports containerized development environments using VS Code Dev Containers or GitHub Codespaces.

This gives a high-level understanding that VS Code's architecture centers on a core open-source editor and extensibility platform, with features and extensions layered on top, and Microsoft’s product builds adding proprietary customizations.

If you want, I can explore specific source code folders or architecture documentation inside the repo to explain more detailed architectural aspects. Would you like me to do that?